In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from ipywidgets import interact_manual
import os
from keras import backend as K
from sklearn.metrics import mean_squared_error
import math
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from keras.utils import to_categorical,Sequence

C:\Users\Andre\Anaconda3\envs\GreatEnv\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
trainDF = pd.read_csv("./input/avito-demand-prediction/train.csv")

trainDF.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


In [3]:
#Handle NA. 
trainDF["description"] = trainDF["description"].fillna("")
trainDF["title"] = trainDF["title"].fillna("")
trainDF = trainDF.fillna(0)

In [4]:
# Create a training and validation set. 
trainDF = trainDF.sample(frac=1)

nItems = trainDF.shape[0]
nValidation = int(nItems * 0.2)

validation = trainDF[0:nValidation]
train      = trainDF[nValidation:]


In [5]:
print(train.shape)
print(validation.shape)

(1202740, 18)
(300684, 18)


In [6]:
categoricalFeats = ["region","city","category_name","parent_category_name","user_type","activation_date"]
quantFeats = ["price","description","title","image_top_1","item_seq_number"]
targetFeat = ["deal_probability"]
feats = categoricalFeats + quantFeats

max_features = 20000
maxlen = 200

def getCatSizes(df,catFeats):
    outDict = {}
    for feat in categoricalFeats:
        outDict[feat] = train[feat].unique().shape[0]
    
    return outDict
        
def preprocessDat(df,catSizes):
    # Convert categorical variables to one-hot vectors. 
    catFeats = []
    
    catFeats.append(to_categorical(pd.Categorical(df["category_name"]).codes,
                                   num_classes=catSizes["category_name"]))
    
    catFeats.append(to_categorical(pd.Categorical(df["city"]).codes,
                                   num_classes=catSizes["city"]))
    
    catFeats.append(to_categorical(pd.Categorical(df["region"]).codes,
                                   num_classes=catSizes["region"]))
    
    catFeats.append(to_categorical(pd.Categorical(df["parent_category_name"]).codes,
                                   num_classes=catSizes["parent_category_name"]))
    
    catFeats.append(to_categorical(pd.Categorical(df["user_type"]).codes,
                                   num_classes=catSizes["user_type"]))
    
    dateTmp = pd.to_datetime(df["activation_date"])
    catFeats.append(pd.get_dummies(dateTmp.map(lambda x: x.dayofweek)).as_matrix())
    
    # Further preprocessing. 
    df["description"] = df["description"].map(len)
    df["title"] = df["title"].map(len)
    
    df = df.drop(categoricalFeats,axis=1)
    
    df = df.replace('', 0, regex=True)
    df = df.fillna(0)
    
    
    return  df,catFeats

# Categorical Category Sizes
catSizes = getCatSizes(trainDF,categoricalFeats)

# Training Preprocessing 
select_train_quant,select_train_categorical = preprocessDat(train[feats],catSizes)

# Validation Preprocesing
select_val_quant,select_val_categorical  = preprocessDat(validation[feats],catSizes)

scaler = preprocessing.RobustScaler().fit(select_train_quant)
select_train_scaled = scaler.transform(select_train_quant)
select_val_scaled   = scaler.transform(select_val_quant)

C:\Users\Andre\Anaconda3\envs\GreatEnv\lib\site-packages\ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Andre\Anaconda3\envs\GreatEnv\lib\site-packages\ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
len(select_train_categorical[2])

1202740

# NN

In [13]:
from keras.layers import Dense, Input,BatchNormalization,Dropout,Concatenate
from keras.models import Model
from keras import backend

def rmse(y_true, y_pred):
    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

layerSize = 1500
quants = Input(shape=(len(quantFeats), ))

catInps = []
for catFeat in select_train_categorical:
    catInps.append(Input(shape=(len(catFeat[0]), )))
    
    
inp   = Concatenate()([quants]+catInps)

x = Dense(layerSize,activation="relu")(inp)
x = Dense(layerSize,activation="relu")(x)
x = Dense(layerSize,activation="relu")(x)
x = Dense(layerSize,activation="relu")(x)
x = Dense(1,activation="sigmoid")(x)

model = Model([quants]+catInps,x)
model.compile(loss="binary_crossentropy",optimizer='RMSProp',metrics=[rmse])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 5)            0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           (None, 47)           0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           (None, 1720)         0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, 28)           0                                            
__________________________________________________________________________________________________
input_19 (

In [16]:
model.fit([select_train_scaled]+select_train_categorical,train["deal_probability"],epochs=100 ,batch_size=100)

Epoch 1/100
 156200/1202740 [==>...........................] - ETA: 2:26 - loss: 2.2441 - rmse: 0.1392

KeyboardInterrupt: 

In [15]:
pred = model.predict([select_val_scaled]+select_val_categorical)
true = validation["deal_probability"]
math.sqrt(mean_squared_error(true,pred))

0.2952683694238856

In [17]:
pred = model.predict([select_train_scaled]+select_train_categorical)
true = train["deal_probability"]
math.sqrt(mean_squared_error(true,pred))

0.29487587846756924

In [17]:
train_mean = select_train["deal_probability"].mean()
pred = np.full(select_val["deal_probability"].shape,train_mean )
true = select_val["deal_probability"]
math.sqrt(mean_squared_error(true,pred))

0.2627653092426415

In [16]:
select_val["deal_probability"].shape

(261417,)